<a href="https://colab.research.google.com/github/nagirishima/Frog_Image/blob/main/Frog_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生成者ネットワーク

In [1]:
import keras
from keras import layers
import numpy as np

In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

#畳み込み層を追加
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#32*32にアップサンプリング
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

#さらに畳み込み層を追加
x= layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x= layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#32*32，1チャネルの特徴マップを生成
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

#generatorモデルをインスタンス化
#形状が(latent_dim,)の入力を形状が(32, 32, 3)の画像にマッピング
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)      

# 判別者ネットワーク

In [3]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

#ドロップアウト層を一つ追加

#分類層
x = layers.Dense(1, activation='sigmoid')(x)

#discriminatorモデルをインスタンス化
#形状が32*32*3の入力で二値分類を実行
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

#オプティマイザで勾配刈り込みを使用し
#訓練を安定させるために学習率減衰を使用(dicay)
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)        

# 敵対者ネットワーク

In [4]:
#discriminatorの重みを訓練不可能に設定(これはganモデルにのみ適用される)
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# GANの訓練の実装

In [8]:
import os
from keras.preprocessing import image
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

#カエルの画像（6）を選択
x_train = x_train[y_train.flatten() == 6]

#データを正則化
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size =20

#生成された画像の保存先を指定
save_dir = '/GAN'

start = 0
for step in range(iterations):
  #潜在空間から点をランダムに抽出
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

  #偽物の画像にデコーディング
  generated_images = generator.predict(random_latent_vectors)

  #本物の画像と組み合わせる
  stop = start + batch_size
  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])

  #本物の画像と偽物の画像を区別するラベルを組み立てる
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

  #ラベルにランダムノイズを追加
  labels += 0.05 * np.random.random(labels.shape)

  #discriminatorを訓練
  d_loss = discriminator.train_on_batch(combined_images, labels)

  #潜在空間から点をランダムに抽出
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

  misleading_targets = np.zeros((batch_size, 1))

  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0

  if step % 100 == 0:
    gan.save_weights('gan.h5')

    print('discriminator loss at step %s: %s' % (step, d_loss))
    print('adversarial loss at step %s: %s' % (step, a_loss))

    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))



discriminator loss at step 0: 0.6930700540542603
adversarial loss at step 0: 0.3704492151737213
discriminator loss at step 100: 0.6892052292823792
adversarial loss at step 100: 0.7605577111244202
discriminator loss at step 200: 0.69190514087677
adversarial loss at step 200: 0.7439895868301392
discriminator loss at step 300: 0.6926822662353516
adversarial loss at step 300: 0.7498670816421509
discriminator loss at step 400: 0.692965567111969
adversarial loss at step 400: 0.7542592883110046
discriminator loss at step 500: 0.6919215321540833
adversarial loss at step 500: 0.7507613897323608
discriminator loss at step 600: 0.6937016248703003
adversarial loss at step 600: 0.7488775253295898
discriminator loss at step 700: 0.696782112121582
adversarial loss at step 700: 0.778291642665863
discriminator loss at step 800: 0.6928269267082214
adversarial loss at step 800: 0.7476600408554077
discriminator loss at step 900: 0.6888861060142517
adversarial loss at step 900: 0.7667003870010376
discrimin